In [24]:
import requests as req
import os
from pathlib import Path

In [21]:
#Config
LIMIT = 100  #limit item per page
PAGE = 5 #limit of page (So it will be LIMIT*PAGE item)

LIMIT_REVIEW = 55
OFF_SET = 55

DATA_PATH = Path("data")

In [4]:
class Item:
    def __init__(self, itemid, shopid):
        self.itemid = itemid
        self.shopid = shopid

class Review:
    def __init__(self, rating, comment, like_count):
        self.rating = rating
        self.comment = comment
        self.like_count = like_count


In [5]:
def getItem(keyword):
    ls_item = []
    page = 0
    while (page < PAGE):
        search_api = f"https://shopee.vn/api/v4/search/search_items?by=relevancy&keyword={keyword}&limit={LIMIT}&newest={page}&order=desc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2"
        response = req.get(search_api)
        items = response.json()
        items = items['items']
        for item in items:
            itemid = item['item_basic']['itemid']
            shopid = item['item_basic']['shopid']
            ls_item.append(Item(itemid, shopid))          
        page+=1
    return ls_item

In [7]:
def getRatingAPI(itemid, shopid, offset):
    api = f"https://shopee.vn/api/v2/item/get_ratings?filter=0&flag=1&itemid={itemid}&limit=55&offset={offset}&shopid={shopid}&type=0"
    return api

In [19]:
def getReview(list_item, outputfile):
    ls_reviews = []
    with open(f"{DATA_PATH}/{outputfile}", 'a') as f:
        
        for item in list_item:
            ls_re = []
            itemid = item.itemid
            shopid = item.shopid
            rating_api = getRatingAPI(itemid, shopid, 0)
            first_rate = req.get(rating_api).json()
            rcount_with_context = first_rate['data']['item_rating_summary']['rcount_with_context']

            offset = 0
            while (offset < rcount_with_context):
                rating_api = getRatingAPI(itemid, shopid, offset)
                response = req.get(rating_api).json()
                try:
                    reviews = response['data']['ratings']
                except:
                    break
                for review in reviews:
                    try:
                        rating = review['rating_star']
                        comment = review['comment'].replace('\n', '.').replace(',', ' ').replace('\r', ' ')
                        comment = ' '.join(comment.split())
                        like_count = review['like_count']
                    except:
                        continue
                    if len(comment):
                        revi = Review(rating, comment, like_count)
                        ls_reviews.append(revi)
                        ls_re.append(revi)
                offset += OFF_SET
            for review in ls_re:
                f.write(f"{review.rating}, {review.comment}, {review.like_count}\n")
        f.close()
    return ls_reviews

In [11]:
ls_items = getItem("quan ao nam")

In [20]:
reviews = getReview(ls_items, "quan_ao_nam.csv")

In [28]:
ls_items = getItem("nuoc hoa")
reviews = getReview(ls_items, "nuoc_hoa.csv")

JSONDecodeError: [Errno Expecting value] <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr><center>nginx</center>
</body>
</html>
: 0